# MNIST en Deep Learning

Dans ce TP, nous allons construire des algorithmes de Deep Learning pour tenter de reconnaître des chiffres manuscrits.

## Chargement des données et transformation

Nous allons travailler sur la base de données MNIST qui contient 60000 images en niveaux de grille de résolution 28x28, représentant les 10 chiffres de 0 à 9, ainsi qu'un jeu de test de 10000 images. Tout d'abord, chargeons ce jeu de données.

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchinfo import summary

import matplotlib
import matplotlib.pyplot as plt
import scipy
import scipy.ndimage
import numpy as np

Téléchargement des données

In [ ]:
train_dataset = datasets.MNIST('../data', train=True, transform=ToTensor()
test_dataset = datasets.MNIST('../data', train=False, transform=ToTensor())

Créer un dataloader et visualiser des images

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=16)


# Parcourez un batch dans le DataLoader
for images, labels in train_dataloader:
    sample_images = images[:5]  # Les 5 premières images
    sample_labels = labels[:5]  # Les 5 premières étiquettes
    break  # On prend seulement le premier batch

# Affichez les images et leurs étiquettes
fig, axes = plt.subplots(1, 5, figsize=(12, 3))

for i, ax in enumerate(axes):
    ax.imshow(
        sample_images[i].squeeze(), cmap="gray"
    )  # .squeeze() pour retirer la dimension du canal
    ax.set_title(f"Label: {sample_labels[i].item()}")
    ax.axis("off")

plt.tight_layout()
plt.show()


## Construire un premier réseau de neurones

Construisons notre premier réseau de neurones.

Pour cela, nous allons créer un modèle Pytorch en utilisant la classe nn.Module vu précedemment:
* __model = Sequential()__

Puis utiliser les méthodes suivantes de Pytroch pour ajouter des couches à ce modèle :

* __nn.FLatten__ : on manipule des vecteurs et non des image, on passe de (28,28) -> (784,)
* __nn.Linear__ : on ajoute une couche dense (ou linéaire)
* __nn.Dropout__ : applique un dropout à la couche, pour éviter le surapprentissage
* __nn.ReLu__ : fonction d'activation relu au coeur du réseau
* __nn.Softmax__ : en sortie de réseau

In [ ]:
# A COMPLETER

# Créer le réseau suivant avec Pytorch :
# 784 dimensions en entrée => 12 dense => activation relu => 12 dense => activation relu => dropout 0.5 => 10 sorties (activation softmax)


class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # à compléter

    def forward(self, x):
        # à compléter
        return x

# Instanciation du modèle
model = MyModel()

# affichage du résumé di modèle
summary(model, imput_size=(28,28,1))

 On commence par créer une boucle d'entrainement qui va faciliter l'apprentissage de notre modèle sur nos données.

In [ ]:
# Défintion de la boucle d'entrainement
def train_loop(
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    loss_fn,
    optimizer,
    epoch: int,
)-> dict:
    start_time = time.time()

    # Apprentissage du modèle
    model.train()  # Met le modèle en mode entraînement
    train_loss, count = 0.0, 0
    for X, y in tqdm(train_dataloader, desc=f"Train epoch {epoch}"):
        X, y = X.to(device), y.to(device)
        
        # Prédiction du réseau de neurones
        y_hat = model(X)  
        # Calcul de l'erreur (y_hat, y)
        loss = loss_fn(y_hat, y)  
        # Backpropagation (MaJ des poids du réseau)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()
        count += X.shape[0]
    train_loss /= count

    # Test du modèle
    model.eval()
    with torch.no_grad():
        test_loss, count = 0.0, 0
        for X, y in tqdm(test_dataloader, desc=f"Test epoch {epoch}"):
            # Déplacer les données vers le dispositif
            X, y = X.to(device), y.to(device)
            # Prédiction du réseau de neurones
            y_hat = model(X)  
            # Calcul de l'erreur (y_hat, y)
            loss = loss_fn(y_hat, y)  
            test_loss += loss.item()
            count += X.shape[0]
    test_loss /= count

    elapsed_time = time.time() - start_time
    print(
        f"Epoch {epoch} | Elapsed Time : {round(elapsed_time, 1)}s | Training Loss : {train_loss:.4f} | Test Loss : {test_loss:.4f}"
    )
    return {"train_loss":train_loss, "test_loss": test_loss}


On définit également les données nécessaires à l'apprentissage

In [ ]:
batch_size = 256
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(train_dataset, batch_size=batch_size)

Ensuite, nous pouvons lancer l'apprentissage des paramètres.

In [ ]:
epochs=20
loss_fn = nn.CrossEntropyLoss()
sgd_optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, nesterov=True)
for epoch in range(epochs):
    model = train_loop(train_dataloader, test_dataloader, model, loss_fn, sgd_optimizer, epoch)

Nous vous laissons analyser les résultats. Ce réseau de neurones est-il performant ?

__A vous de jouer__ : essayez de créer un meilleur réseau de neurones, afin d'atteindre le meilleur résultat possible.

In [ ]:
# Créer un meilleur réseau de neurones, et l'entraîner
# Objectif : avoir le meilleur résultat possible

# Nous ne donnons pas la correction. Il y a plusieurs réponses possibles.
# Vous pouvez par exemple ajouter des couches, modifier le nombre de neurones par couche et jouer sur le dropout.

Voyons ce que donne notre modèle sur un exemple.

In [ ]:
def plot_mnist_digit(image):
    """ Plot a single MNIST image."""
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(image, cmap = matplotlib.cm.binary)
    plt.xticks(np.array([]))
    plt.yticks(np.array([]))
    plt.show()
loss,acc = model.evaluate(X_test, y_test,  verbose=0)
index=800
print('The accuracy on the test set is ',(acc*100),'%')
plot_mnist_digit(X_test_base[index])
#cl=model.predict_classes(X_test[index].reshape((1,784)))
cl = np.argmax(model.predict(X_test[index].reshape((1,784))), axis=-1)

print("le chiffre reconnu est: ", cl[0])
print("le chiffre à reconnaitre  est: ", np.argmax(y_test[index]))

## CNN : réseaux de neurones convolutionnels

Nous allons maintenant implémenter un réseau de neurones convolutionnel.

Pour cet exercice, vous allez avoir besoin des méthodes Pytorch suivantes, en plus de celles déjà vues précédemment :

Construisons notre premier réseau de neurones.

* __nn.Conv2d__ : on ajoute une couche de convolution
* __nn.MaxPool2d__ : fonction de max pooling qui permet de réduire la dimension des images d'entrée

In [ ]:
# Définition du modèle
class MyModel(nn.Module):
    def __init__(self, input_shape, nb_classes):
        super(MyModel, self).__init__()
        # à compléter 

    def forward(self, x):
        # à compléter 

        return x
        
# Instanciation du modèle
model = MyModel()

# affichage du résumé di modèle
summary(model, imput_size=())

In [ ]:
batch_size = 256
epochs=20
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,  verbose=1, validation_data=(x_test, y_test))

In [ ]:
def plot_mnist_digit(image):
    """ Plot a single MNIST image."""
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(image, cmap = matplotlib.cm.binary)
    plt.xticks(np.array([]))
    plt.yticks(np.array([]))
    plt.show()
loss,acc = model.evaluate(x_test, y_test,  verbose=0)
index=800
print('The accuracy on the test set is ',(acc*100),'%')
plot_mnist_digit(X_test_base[index])
#cl=model.predict_classes(x_test[index].reshape((1,28,28,1)))
cl = np.argmax(model.predict(X_test[index].reshape((1,28,28,1))), axis=-1)


print("le chiffre reconnu est: ", cl[0])
print("le chiffre à reconnaitre  est: ", np.argmax(y_test[index]))


In [ ]:
# Créer un meilleur réseau de neurones convolutionnel, et l'entraîner
# Objectif : avoir le meilleur résultat possible

# Nous ne donnons pas la correction. Il y a plusieurs réponses possibles.
# Vous pouvez par exemple ajouter des couches convolutionnelles et max_pooling,
# modifier le nombre de convolutions ou leur taille et jouer sur le dropout.

## Bonus : Auto encodeur

L'auto-encodeur est un réseau de neurones qui compresse puis décompresse l'information. On l'entraîne en lui demandant de retrouver en sortie la même image que celle qu'il avait en entrée. Ici, l'information en entrée est en dimension 784 (28x28), et l'auto-encodeur va la compresser en dimension 2.

In [ ]:
encoding_dim = 2

model = Sequential()
model.add(Dense(encoding_dim, input_shape=(784,),activation='relu'))

model.add(Dense(784, activation='sigmoid'))

# Définition du modèle
class MyAutoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.input_shape = (784,)

        # Couche d'encodage
        self.encoder = nn.Linear(input_shape, encoding_dim)
        self.relu = nn.ReLU()

        # Couche de décodage
        self.decoder = nn.Linear(encoding_dim, input_shape)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Encodage
        x = self.relu(self.encoder(x))

        # Décodage
        x = self.sigmoid(self.decoder(x))

        return x

model= MyAutoencoder()

model.summary()



In [ ]:
decoded_imgs = model.predict(X_test)

Affichons quelques images pour voir comment se comporte notre auto-encodeur.

In [ ]:
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

__A vous de jouer__ : essayez d'améliorer l'auto-encodeur.

In [ ]:
## A COMPLETER

# Nous ne donnons pas la correction. Il y a plusieurs réponses possibles.
# Vous pouvez par exemple ajouter des couches ou modifier le nombre de neurones de la couche cachée.